In [11]:
#Loading Data
import os
from pathlib import Path
import shutil

# Define paths
original_dataset_dir = "../Project4/images/Images"
base_dir = "../Project4/split_data"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
#Validation set for overfitting of complex models
val_dir = os.path.join(base_dir, "val")
# Ensure directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Define classes
classes = os.listdir(original_dataset_dir)

In [12]:
# Create directories for each class in train and test directories
for class_name in classes:
    class_train_dir = os.path.join(train_dir, class_name)
    class_test_dir = os.path.join(test_dir, class_name)
    class_val_dir = os.path.join(val_dir, class_name)
    os.makedirs(class_train_dir, exist_ok=True)
    os.makedirs(class_test_dir, exist_ok=True)
    os.makedirs(class_val_dir, exist_ok=True)

In [13]:
from random import shuffle

split_ratio_train = 0.7  # 70% train
split_ratio_val = 0.15  # 15% validation (from train data)
split_ratio_test = 0.15  # 15% test

for class_name in classes:
  class_images = os.listdir(os.path.join(original_dataset_dir, class_name))
  shuffle(class_images)  # Randomize image order

  num_images = len(class_images)

  # Calculate split points (adjust ratios as needed)
  num_train_images = int(split_ratio_train * num_images)
  num_val_images = int(split_ratio_val * num_images)
  num_test_images = num_images - num_train_images - num_val_images

  # Train set
  for image_name in class_images[:num_train_images]:
    src = os.path.join(original_dataset_dir, class_name, image_name)
    dst = os.path.join(train_dir, class_name, image_name)
    shutil.copyfile(src, dst)

  # Validation set (from train data)
  for image_name in class_images[num_train_images:num_train_images+num_val_images]:
    src = os.path.join(original_dataset_dir, class_name, image_name)
    dst = os.path.join(val_dir, class_name, image_name)
    shutil.copyfile(src, dst)

  # Test set
  for image_name in class_images[num_train_images+num_val_images:]:
    src = os.path.join(original_dataset_dir, class_name, image_name)
    dst = os.path.join(test_dir, class_name, image_name)
    shutil.copyfile(src, dst)

print("Data split and directories created successfully.")

Data split and directories created successfully.


In [14]:
#Check the splits worked
# Count the total number of images in train and test directories
#Using underscores to save memory by not storing labels,path, or directories and simply count number of files in each sub directory
total_train_count = sum(len(files) for _, _, files in os.walk(train_dir))
total_test_count = sum(len(files) for _, _, files in os.walk(test_dir))
total_val_count = sum(len(files) for _, _, files in os.walk(val_dir))
total_images = total_test_count+total_train_count+total_val_count
train_per = total_train_count/total_images *100
test_per = total_test_count/total_images*100
val_per = total_val_count/total_images *100

print(f"Total Images: {total_images}")
print(f"Total Train Images: {total_train_count}, Percent of Total: {train_per}")
print(f"Total Test Images: {total_test_count}, Percent of Total: {test_per}")
print(f"Total Validation Images: {total_val_count}, Percent of Total: {val_per}")
print("Split is Effective!")
classes_val = os.listdir(val_dir)
classes_test = os.listdir(test_dir)
classes_train = os.listdir(train_dir)

print(len(classes_val))
print(len(classes_test))
print(len(classes_train))

Total Images: 20580
Total Train Images: 14355, Percent of Total: 69.75218658892129
Total Test Images: 3200, Percent of Total: 15.54907677356657
Total Validation Images: 3025, Percent of Total: 14.698736637512146
Split is Effective!
120
120
120


In [19]:
#Since we are no longer using simple sequential models and are Using the Functional API we rescale with generators
train_datagen = ImageDataGenerator(rescale=1./255,  # Rescale images to [0, 1]
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)  # Example augmentations

val_datagen = ImageDataGenerator(rescale=1./255)  # Rescale for validation

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Adjust image size as needed
    batch_size=32,
    class_mode='categorical'  # Assuming categorical classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 14355 images belonging to 120 classes.
Found 3025 images belonging to 120 classes.


In [20]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Add, Input, Activation

In [21]:
#Now on to training our Models!

In [22]:
def residual_block(x, filters):
  # Shortcut connection
  shortcut = x

  # Convolutional layers with Batch Normalization
  x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding="same")(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)  # ReLU activation

  x = Conv2D(filters, kernel_size=(3, 3), strides=1, padding="same")(x)
  x = BatchNormalization()(x)

  # Add shortcut connection with element-wise sum
  x = Add()([x, shortcut])
  x = Activation('relu')(x)  # ReLU activation after adding shortcut
  return x

In [25]:
#ResNet Model
inputs = Input(shape=(224, 224, 3))  # Input image shape

#Initial convolutional layer 64 filters, 7x7
x = Conv2D(64, kernel_size=(7, 7), strides=2, padding="same")(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)  # ReLU activation

#Max pooling
x = MaxPooling2D(pool_size=(3, 3), strides=2)(x)

#**Residual block 1**
x = residual_block(x, 64)  # Replace 64 with desired number of filters

#**Residual block 2**
# You can add more residual blocks here (e.g., residual_block(x, 128))

# Global average pooling
x = GlobalAveragePooling2D()(x)

# Flatten and fully-connected layer
x = Flatten()(x)
outputs = Dense(120, activation='softmax')(x)  # Change number of neurons for your classification task

# Create the model using Model for more complex ResNet network
ResNet = Model(inputs=inputs, outputs=outputs)


# Compile the model (adjust optimizer and loss as needed)
ResNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

ResNet.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_14 (Conv2D)          (None, 112, 112, 64)         9472      ['input_4[0][0]']             
                                                                                                  
 batch_normalization_9 (Bat  (None, 112, 112, 64)         256       ['conv2d_14[0][0]']           
 chNormalization)                                                                                 
                                                                                                  
 activation_9 (Activation)   (None, 112, 112, 64)         0         ['batch_normalization_9[

In [ ]:
#fit the model from image generator
history = ResNet.fit(
            train_generator,
            batch_size=32,
            epochs=20,
            validation_data=val_generator
)

Epoch 1/20
206/449 [============>.................] - ETA: 2:09 - loss: 4.8116 - accuracy: 0.0137

In [ ]:
#We can directly import from Keras here
from keras.applications.vgg16 import VGG16
model_vgg16 = VGG16(weights='imagenet')
model_vgg16.summary()

In [ ]:
#fit the model from image generator
history = model_vgg16.fit(
            train_generator,
            batch_size=32,
            epochs=20,
            validation_data=val_generator
)

In [8]:
#Model AlexNet
AlexNet = models.Sequential()

# Layer 1: Convolutional layer with 96 filters of size 11x11, followed by max pooling
AlexNet.add(layers.Conv2D(96, kernel_size=(11,11),strides=4, activation='relu', input_shape=(224, 224, 3)))
AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

AlexNet.add(layers.Conv2D(256, kernel_size=(5,5),padding='same', activation='relu'))
AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

AlexNet.add(layers.Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
AlexNet.add(layers.Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
AlexNet.add(layers.Conv2D(256, kernel_size=(3,3),padding='same', activation='relu'))

AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

# Flatten the feature maps to feed into fully connected layers
AlexNet.add(layers.Flatten())


# Layer 3: Fully connected layer with 120 neurons
AlexNet.add(layers.Dense(4096, activation='relu'))
AlexNet.add(layers.Dropout(.5))
# Layer 4: Fully connected layer with 84 neurons
AlexNet.add(layers.Dense(4096, activation='relu'))
AlexNet.add(layers.Dropout(.5))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
AlexNet.add(layers.Dense(120, activation='softmax'))

# Compile model
AlexNet.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 26, 26, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 384)       1

In [9]:
#fit the model from image generator
history = AlexNet.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
411/411 [==============================] - 309s 749ms/step - loss: 4.7867 - accuracy: 0.0090 - val_loss: 4.7836 - val_accuracy: 0.0113
Epoch 2/20
411/411 [==============================] - 304s 740ms/step - loss: 4.7608 - accuracy: 0.0124 - val_loss: 4.6992 - val_accuracy: 0.0198
Epoch 3/20
411/411 [==============================] - 310s 754ms/step - loss: 4.6111 - accuracy: 0.0281 - val_loss: 4.6498 - val_accuracy: 0.0219
Epoch 4/20
411/411 [==============================] - 309s 750ms/step - loss: 4.4431 - accuracy: 0.0373 - val_loss: 4.3622 - val_accuracy: 0.0460
Epoch 5/20
411/411 [==============================] - 307s 747ms/step - loss: 4.3068 - accuracy: 0.0495 - val_loss: 4.2653 - val_accuracy: 0.0518
Epoch 6/20
411/411 [==============================] - 308s 748ms/step - loss: 4.1833 - accuracy: 0.0588 - val_loss: 4.2922 - val_accuracy: 0.0515
Epoch 7/20
411/411 [==============================] - 308s 749ms/step - loss: 4.0663 - accuracy: 0.0762 - val_loss: 4.1679 -

In [ ]:
#We can see major overfitting from epochs 11 on.